# Transformer 网络

欢迎来到第 4 周的作业，这是深度学习专项课程（Deep Learning Specialization）第 5 门课程的最后一个作业！  
同时，恭喜你走到整个深度学习专项课程的最后一份作业——你已经接近完成整个课程了！

在课程的前面部分，你已经实现了顺序神经网络（sequential neural networks），如 RNN、GRU 和 LSTM。  
在本笔记中，你将探索 **Transformer 架构**，这是一种可以利用并行处理并显著加速训练过程的神经网络。

**完成本作业后，你将能够：**

* 创建位置编码（positional encodings）以捕捉数据中的序列关系
* 使用词向量计算缩放点积自注意力（scaled dot-product self-attention）
* 实现掩码多头注意力（masked multi-head attention）
* 构建并训练一个 Transformer 模型

最后一次，让我们开始吧！


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# UNQ_C1: get_angles
# -------------------------------
def get_angles(pos, k, d):
    """
    Get the angles for the positional encoding
    """
    i = k
    angles = pos / np.power(10000, (2 * (i // 2)) / np.float32(d))
    return angles

# -------------------------------
# get_angles 测试
# -------------------------------
pos_m = np.arange(4)[:, np.newaxis]
dims = np.arange(8)[np.newaxis, :]
angles = get_angles(pos_m, dims, 8)
assert angles.shape == (4, 8)
print("get_angles passed!")

# -------------------------------
# UNQ_C2: positional_encoding
# -------------------------------
def positional_encoding(positions, d):
    angle_rads = get_angles(np.arange(positions)[:, np.newaxis],
                            np.arange(d)[np.newaxis, :],
                            d)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return torch.tensor(pos_encoding, dtype=torch.float32)

# -------------------------------
# positional_encoding 测试
# -------------------------------
pos_encoding = positional_encoding(50, 512)
assert pos_encoding.shape == (1, 50, 512)
print("positional_encoding passed!")

# 可视化
# plt.pcolormesh(pos_encoding[0], cmap='RdBu')
# plt.xlabel('d')
# plt.ylabel('Position')
# plt.colorbar()
# plt.show()

# -------------------------------
# UNQ_C3: scaled_dot_product_attention
# -------------------------------
def scaled_dot_product_attention(q, k, v, mask=None):
    matmul_qk = torch.matmul(q, k.transpose(-2, -1))
    dk = k.size(-1)
    scaled_attention_logits = matmul_qk / np.sqrt(dk)
    if mask is not None:
        scaled_attention_logits = scaled_attention_logits.masked_fill(mask == 0, float('-1e9'))
    attention_weights = F.softmax(scaled_attention_logits, dim=-1)
    output = torch.matmul(attention_weights, v)
    return output, attention_weights

# -------------------------------
# scaled_dot_product_attention 测试
# -------------------------------
q = torch.rand(1, 3, 4)
k = torch.rand(1, 3, 4)
v = torch.rand(1, 3, 4)
out, attn = scaled_dot_product_attention(q, k, v)
assert out.shape == (1, 3, 4)
assert attn.shape == (1, 3, 3)
print("scaled_dot_product_attention passed!")

# -------------------------------
# FullyConnected
# -------------------------------
def FullyConnected(embedding_dim, fully_connected_dim):
    return nn.Sequential(
        nn.Linear(embedding_dim, fully_connected_dim),
        nn.ReLU(),
        nn.Linear(fully_connected_dim, embedding_dim)
    )

# -------------------------------
# UNQ_C4: EncoderLayer
# -------------------------------
class EncoderLayer(nn.Module):
    def __init__(self, embedding_dim, num_heads, fully_connected_dim, dropout_rate=0.1, layernorm_eps=1e-6):
        super().__init__()
        self.mha = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads, batch_first=True)
        self.ffn = FullyConnected(embedding_dim, fully_connected_dim)
        self.layernorm1 = nn.LayerNorm(embedding_dim, eps=layernorm_eps)
        self.layernorm2 = nn.LayerNorm(embedding_dim, eps=layernorm_eps)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        attn_output, _ = self.mha(x, x, x)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

# -------------------------------
# EncoderLayer 测试
# -------------------------------
x = torch.rand(2, 5, 32)
layer = EncoderLayer(32, 4, 64)
out = layer(x)
assert out.shape == (2, 5, 32)
print("EncoderLayer passed!")

# -------------------------------
# UNQ_C5: Encoder
# -------------------------------
class Encoder(nn.Module):
    def __init__(self, num_layers, embedding_dim, num_heads, fully_connected_dim, input_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super().__init__()
        self.embedding = nn.Embedding(input_vocab_size, embedding_dim)
        self.pos_encoding = positional_encoding(maximum_position_encoding, embedding_dim)
        self.layers = nn.ModuleList([EncoderLayer(embedding_dim, num_heads, fully_connected_dim, dropout_rate) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        seq_len = x.size(1)
        x = self.embedding(x) * np.sqrt(self.embedding.embedding_dim)
        x = x + self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x)
        for layer in self.layers:
            x = layer(x)
        return x

# -------------------------------
# Encoder 测试
# -------------------------------
x = torch.randint(0, 50, (2, 10))
enc = Encoder(num_layers=2, embedding_dim=32, num_heads=4, fully_connected_dim=64, input_vocab_size=50, maximum_position_encoding=100)
out = enc(x)
assert out.shape == (2, 10, 32)
print("Encoder passed!")

# -------------------------------
# UNQ_C6: DecoderLayer
# -------------------------------
class DecoderLayer(nn.Module):
    def __init__(self, embedding_dim, num_heads, fully_connected_dim, dropout_rate=0.1):
        super().__init__()
        self.mha1 = nn.MultiheadAttention(embedding_dim, num_heads, batch_first=True)
        self.mha2 = nn.MultiheadAttention(embedding_dim, num_heads, batch_first=True)
        self.ffn = FullyConnected(embedding_dim, fully_connected_dim)
        self.layernorm1 = nn.LayerNorm(embedding_dim)
        self.layernorm2 = nn.LayerNorm(embedding_dim)
        self.layernorm3 = nn.LayerNorm(embedding_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, enc_output):
        attn1, attn_w1 = self.mha1(x, x, x)
        x1 = self.layernorm1(x + attn1)
        attn2, attn_w2 = self.mha2(x1, enc_output, enc_output)
        x2 = self.layernorm2(x1 + attn2)
        ffn_output = self.ffn(x2)
        ffn_output = self.dropout(ffn_output)
        out = self.layernorm3(x2 + ffn_output)
        return out, attn_w1, attn_w2

# -------------------------------
# DecoderLayer 测试
# -------------------------------
x = torch.rand(2, 5, 32)
enc_out = torch.rand(2, 6, 32)
dec_layer = DecoderLayer(32, 4, 64)
out, a1, a2 = dec_layer(x, enc_out)
assert out.shape == (2, 5, 32)
print("DecoderLayer passed!")

# -------------------------------
# UNQ_C7: Decoder
# -------------------------------
class Decoder(nn.Module):
    def __init__(self, num_layers, embedding_dim, num_heads, fully_connected_dim, target_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super().__init__()
        self.embedding = nn.Embedding(target_vocab_size, embedding_dim)
        self.pos_encoding = positional_encoding(maximum_position_encoding, embedding_dim)
        self.layers = nn.ModuleList([DecoderLayer(embedding_dim, num_heads, fully_connected_dim, dropout_rate) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, enc_output):
        seq_len = x.size(1)
        x = self.embedding(x) * np.sqrt(self.embedding.embedding_dim)
        x = x + self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x)
        attn_weights = {}
        for i, layer in enumerate(self.layers):
            x, attn1, attn2 = layer(x, enc_output)
            attn_weights[f'decoder_layer{i+1}_block1'] = attn1
            attn_weights[f'decoder_layer{i+1}_block2'] = attn2
        return x, attn_weights

# -------------------------------
# Decoder 测试
# -------------------------------
x = torch.randint(0, 60, (2, 5))
enc_out = torch.rand(2, 6, 32)
dec = Decoder(num_layers=2, embedding_dim=32, num_heads=4, fully_connected_dim=64, target_vocab_size=60, maximum_position_encoding=100)
out, attn = dec(x, enc_out)
assert out.shape == (2, 5, 32)
print("Decoder passed!")

# -------------------------------
# UNQ_C8: Transformer
# -------------------------------
class Transformer(nn.Module):
    def __init__(self, num_layers, embedding_dim, num_heads, fully_connected_dim,
                 input_vocab_size, target_vocab_size, max_pos_enc_input, max_pos_enc_target, dropout_rate=0.1):
        super().__init__()
        self.encoder = Encoder(num_layers, embedding_dim, num_heads, fully_connected_dim,
                               input_vocab_size, max_pos_enc_input, dropout_rate)
        self.decoder = Decoder(num_layers, embedding_dim, num_heads, fully_connected_dim,
                               target_vocab_size, max_pos_enc_target, dropout_rate)
        self.final_layer = nn.Linear(embedding_dim, target_vocab_size)

    def forward(self, inp, tar):
        enc_output = self.encoder(inp)
        dec_output, attn_weights = self.decoder(tar, enc_output)
        final_output = F.softmax(self.final_layer(dec_output), dim=-1)
        return final_output, attn_weights

# -------------------------------
# Transformer 测试
# -------------------------------
inp = torch.randint(0, 50, (2, 10))
tar = torch.randint(0, 60, (2, 8))
model = Transformer(num_layers=2, embedding_dim=32, num_heads=4, fully_connected_dim=64,
                    input_vocab_size=50, target_vocab_size=60,
                    max_pos_enc_input=100, max_pos_enc_target=100)

final_output, attn = model(inp, tar)
assert final_output.shape == (2, 8, 60)
print("Transformer passed!")


get_angles passed!
positional_encoding passed!
scaled_dot_product_attention passed!
EncoderLayer passed!
Encoder passed!
DecoderLayer passed!
Decoder passed!
Transformer passed!


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np

# ----------------------------
# 位置编码
# ----------------------------
def get_angles(pos, i, d_model):
    return pos / torch.pow(10000, (2 * (i//2)) / d_model)

def positional_encoding(positions, d_model):
    angle_rads = get_angles(
        torch.arange(positions).unsqueeze(1).float(),
        torch.arange(d_model).unsqueeze(0).float(),
        d_model
    )
    # sin on even indices; cos on odd indices
    angle_rads[:, 0::2] = torch.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = torch.cos(angle_rads[:, 1::2])
    return angle_rads.unsqueeze(0)  # (1, positions, d_model)

# 测试位置编码
pos_encoding = positional_encoding(50, 512)
assert pos_encoding.shape == (1, 50, 512)
print("get_angles & positional_encoding passed!")

# ----------------------------
# Mask
# ----------------------------
def create_padding_mask(seq):
    mask = (seq == 0).float()
    return mask.unsqueeze(1)  # (batch_size, 1, seq_len)

def create_look_ahead_mask(size):
    mask = torch.triu(torch.ones(size, size), diagonal=1)
    return mask  # (seq_len, seq_len)

# 测试 Mask
x = torch.tensor([[7, 6, 0, 0, 1],[1, 2, 3, 0, 0],[0, 0, 0, 4, 5]])
pad_mask = create_padding_mask(x)
look_mask = create_look_ahead_mask(5)
assert pad_mask.shape == (3,1,5)
assert look_mask.shape == (5,5)
print("Mask functions passed!")

# ----------------------------
# Scaled Dot-Product Attention
# ----------------------------
def scaled_dot_product_attention(q, k, v, mask=None):
    dk = q.size(-1)
    scores = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(dk)
    if mask is not None:
        scores = scores.masked_fill(mask==1, float('-inf'))
    attn = F.softmax(scores, dim=-1)
    output = torch.matmul(attn, v)
    return output, attn

# 测试 attention
q = k = v = torch.rand(2,3,4)
out, attn = scaled_dot_product_attention(q,k,v)
assert out.shape == (2,3,4) and attn.shape == (2,3,3)
print("Scaled Dot-Product Attention passed!")

# ----------------------------
# 前馈网络
# ----------------------------
class FullyConnected(nn.Module):
    def __init__(self, d_model, dff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, dff)
        self.linear2 = nn.Linear(dff, d_model)
    def forward(self, x):
        return self.linear2(F.relu(self.linear1(x)))

# 测试前馈网络
fc = FullyConnected(8, 16)
x = torch.rand(2,5,8)
y = fc(x)
assert y.shape == x.shape
print("FullyConnected passed!")

# ----------------------------
# Encoder Layer
# ----------------------------
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, dropout=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        self.ffn = FullyConnected(d_model, dff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, mask=None):
        attn_out,_ = self.mha(x,x,x, key_padding_mask=mask)
        x = self.norm1(x + attn_out)
        ffn_out = self.dropout(self.ffn(x))
        x = self.norm2(x + ffn_out)
        return x

# 测试 EncoderLayer
enc_layer = EncoderLayer(8,2,16)
x = torch.rand(2,5,8)
mask = torch.zeros(2,5)
out = enc_layer(x, mask)
assert out.shape == x.shape
print("EncoderLayer passed!")

# ----------------------------
# Encoder
# ----------------------------
class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, max_pos_enc):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(max_pos_enc, d_model)
        self.layers = nn.ModuleList([EncoderLayer(d_model,num_heads,dff) for _ in range(num_layers)])
        self.dropout = nn.Dropout(0.1)
    def forward(self, x, mask=None):
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = x + self.pos_encoding[:,:x.size(1), :]
        x = self.dropout(x)
        for layer in self.layers:
            x = layer(x, mask)
        return x

# 测试 Encoder
encoder = Encoder(2,8,2,16,50,100)
x_inp = torch.randint(0,50,(2,10))
out = encoder(x_inp)
assert out.shape == (2,10,8)
print("Encoder passed!")

# ----------------------------
# Decoder Layer
# ----------------------------
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff):
        super().__init__()
        self.mha1 = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        self.mha2 = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        self.ffn = FullyConnected(d_model,dff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)
    def forward(self, x, enc_output, look_ahead_mask=None, padding_mask=None):
        attn1,_ = self.mha1(x,x,x,key_padding_mask=look_ahead_mask)
        x = self.norm1(x + attn1)
        attn2,_ = self.mha2(x, enc_output, enc_output, key_padding_mask=padding_mask)
        x = self.norm2(x + attn2)
        ffn_out = self.dropout(self.ffn(x))
        x = self.norm3(x + ffn_out)
        return x

# 测试 DecoderLayer
dec_layer = DecoderLayer(8,2,16)
enc_out = torch.rand(2,10,8)
x_tar = torch.rand(2,5,8)
out = dec_layer(x_tar, enc_out)
assert out.shape == x_tar.shape
print("DecoderLayer passed!")

# ----------------------------
# Decoder
# ----------------------------
class Decoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, max_pos_enc):
        super().__init__()
        self.embedding = nn.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(max_pos_enc, d_model)
        self.layers = nn.ModuleList([DecoderLayer(d_model,num_heads,dff) for _ in range(num_layers)])
        self.dropout = nn.Dropout(0.1)
    def forward(self, x, enc_output):
        x = self.embedding(x) * math.sqrt(self.embedding.embedding_dim)
        x = x + self.pos_encoding[:,:x.size(1), :]
        x = self.dropout(x)
        for layer in self.layers:
            x = layer(x, enc_output)
        return x

# 测试 Decoder
decoder = Decoder(2,8,2,16,50,100)
x_tar = torch.randint(0,50,(2,5))
out = decoder(x_tar, enc_out)
assert out.shape == (2,5,8)
print("Decoder passed!")

# ----------------------------
# Transformer
# ----------------------------
class Transformer(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, max_pos_enc_input, max_pos_enc_target):
        super().__init__()
        self.encoder = Encoder(num_layers,d_model,num_heads,dff,input_vocab_size,max_pos_enc_input)
        self.decoder = Decoder(num_layers,d_model,num_heads,dff,target_vocab_size,max_pos_enc_target)
        self.final_layer = nn.Linear(d_model,target_vocab_size)
    def forward(self, inp, tar):
        enc_out = self.encoder(inp)
        dec_out = self.decoder(tar, enc_out)
        return F.softmax(self.final_layer(dec_out), dim=-1)

# 测试 Transformer
model = Transformer(2,8,2,16,50,50,100,100)
x_inp = torch.randint(0,50,(2,10))
x_tar = torch.randint(0,50,(2,5))
out = model(x_inp,x_tar)
assert out.shape == (2,5,50)
print("Transformer passed! Forward output shape:", out.shape)


get_angles & positional_encoding passed!
Mask functions passed!
Scaled Dot-Product Attention passed!
FullyConnected passed!
EncoderLayer passed!
Encoder passed!
DecoderLayer passed!
Decoder passed!
Transformer passed! Forward output shape: torch.Size([2, 5, 50])


## 目录

- [Packages](#0) 软件包
- [1 - 位置编码 (Positional Encoding)](#1)
    - [1.1 - 正弦和余弦角度 (Sine and Cosine Angles)](#1-1)
        - [练习 1 - get_angles](#ex-1)
    - [1.2 - 正弦和余弦位置编码 (Sine and Cosine Positional Encodings)](#1-2)
        - [练习 2 - positional_encoding](#ex-2)
- [2 - Masking 掩码](#2)
    - [2.1 - 填充掩码 (Padding Mask)](#2-1)
    - [2.2 - 前瞻掩码 (Look-ahead Mask)](#2-2)
- [3 - 自注意力 (Self-Attention)](#3)
    - [练习 3 - scaled_dot_product_attention](#ex-3)
- [4 - 编码器 (Encoder)](#4)
    - [4.1 编码器层 (Encoder Layer)](#4-1)
        - [练习 4 - EncoderLayer](#ex-4)
    - [4.2 完整编码器 (Full Encoder)](#4-2)
        - [练习 5 - Encoder](#ex-5)
- [5 - 解码器 (Decoder)](#5)
    - [5.1 解码器层 (Decoder Layer)](#5-1)
        - [练习 6 - DecoderLayer](#ex-6)
    - [5.2 完整解码器 (Full Decoder)](#5-2)
        - [练习 7 - Decoder](#ex-7)
- [6 - Transformer](#6)
    - [练习 8 - Transformer](#ex-8)
- [7 - 参考文献 (References)](#7)


<a name='0'></a>
## 软件包 (Packages)

运行以下代码单元以加载所需的软件包。


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np


<a name='1'></a>
## 1 - 位置编码 (Positional Encoding)

在序列到序列（sequence-to-sequence）任务中，数据的相对顺序对其含义非常重要。  
当你训练顺序神经网络（如 RNN）时，输入数据按顺序传入网络，模型自动获得数据顺序的信息。  

然而，当使用多头注意力（multi-head attention）训练 Transformer 网络时，你会一次性将所有数据输入模型。  
虽然这样大大减少了训练时间，但模型无法直接获取数据顺序信息。  

这时，**位置编码（positional encoding）**就非常有用——你可以专门对输入位置进行编码，并使用以下正弦和余弦公式将其传入网络：
    
$$
PE_{(pos, 2i)}= \sin\left(\frac{pos}{{10000}^{\frac{2i}{d}}}\right)
\tag{1}$$
<br>
$$
PE_{(pos, 2i+1)}= \cos\left(\frac{pos}{{10000}^{\frac{2i}{d}}}\right)
\tag{2}$$

* $d$ 为词向量（word embedding）和位置编码的维度
* $pos$ 为单词的位置
* $i$ 表示位置编码的各个维度

为了理解位置编码，你可以将其看作包含单词相对位置信息的特征。  
最终，**词向量与位置编码的和**会被输入模型。  

如果你只是硬编码位置，比如将一个全 1 或整数矩阵直接加到词向量上，会扭曲语义信息。  
而正弦和余弦函数的值在 [-1, 1] 之间，加入到词向量后，不会显著扭曲词向量，而是为其增加了位置信息。  
使用这两个公式的组合，可以帮助 Transformer 网络关注输入数据的相对位置。  

> 这只是对位置编码的简要介绍，若想进一步理解，可以参考 *Positional Encoding Ungraded Lab*。

**注意：** 在课程讲解中 Andrew 使用的是竖向向量，但本作业中所有向量均为横向。所有矩阵乘法请相应调整。

<a name='1-1'></a>
### 1.1 - 正弦与余弦角度 (Sine and Cosine Angles)

注意，尽管正弦和余弦位置编码公式输入的参数不同（`2i` 与 `2i+1`，偶数与奇数），但公式的内部项相同：
$$\theta(pos, i, d) = \frac{pos}{10000^{\frac{2i}{d}}} \tag{3}$$

在计算序列中某个单词的位置编码时，可以参考内部项：  
$PE_{(pos, 0)}= \sin\left(\frac{pos}{{10000}^{0/d}}\right)$，因为 `2i = 0` 得 `i = 0`  
$PE_{(pos, 1)}= \cos\left(\frac{pos}{{10000}^{0/d}}\right)$，因为 `2i + 1 = 1` 得 `i = 0`  

两者的角度相同！  
$PE_{(pos, 2)}$ 和 $PE_{(pos, 3)}$ 的角度也相同，因为 `i = 1`，所以内部项为 $\frac{pos}{{10000}^{1/d}}$。  
这种关系对所有成对的正弦和余弦曲线都成立：

|      k         | <code>0</code> | <code>1</code> | <code>2</code> | <code>3</code> | <code>...</code> | <code>d-2</code> | <code>d-1</code> |
|----------------|:---------------:|:---------------:|:---------------:|:---------------:|:----------------:|:----------------:|:----------------:|
| encoding(0) =  | [$\sin(\theta(0,0,d))$ | $\cos(\theta(0,0,d))$ | $\sin(\theta(0,1,d))$ | $\cos(\theta(0,1,d))$ | ... | $\sin(\theta(0,d//2,d))$ | $\cos(\theta(0,d//2,d))$] |
| encoding(1) =  | [$\sin(\theta(1,0,d))$ | $\cos(\theta(1,0,d))$ | $\sin(\theta(1,1,d))$ | $\cos(\theta(1,1,d))$ | ... | $\sin(\theta(1,d//2,d))$ | $\cos(\theta(1,d//2,d))$] |
| ...            | ...             | ...             | ...             | ...             | ...              | ...              | ...              |
| encoding(pos) =| [$\sin(\theta(pos,0,d))$ | $\cos(\theta(pos,0,d))$ | $\sin(\theta(pos,1,d))$ | $\cos(\theta(pos,1,d))$ | ... | $\sin(\theta(pos,d//2,d))$ | $\cos(\theta(pos,d//2,d))$] |

<a name='ex-1'></a>
### 练习 1 - get_angles

实现 `get_angles()` 函数，用于计算正弦和余弦位置编码可能的角度。

**提示：**

- 如果 `k = [0, 1, 2, 3, 4, 5]`，那么 `i` 应为 `i = [0, 0, 1, 1, 2, 2]`
- `i = k//2`


In [2]:
# ----------------------------
# 位置编码
# ----------------------------
def get_angles(pos, i, d_model):
    return pos / torch.pow(10000, (2 * (i//2)) / d_model)



In [5]:
# ----------------------------
# Example
# ----------------------------
position = 4
d_model = 8
pos_m = torch.arange(position).unsqueeze(1)  # shape (4,1)
dims = torch.arange(d_model).unsqueeze(0)    # shape (1,8)
angles = get_angles(pos_m, dims, d_model)
print("Angles shape:", angles.shape)
print(angles)

Angles shape: torch.Size([4, 8])
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e-01, 1.0000e-01, 1.0000e-02, 1.0000e-02,
         1.0000e-03, 1.0000e-03],
        [2.0000e+00, 2.0000e+00, 2.0000e-01, 2.0000e-01, 2.0000e-02, 2.0000e-02,
         2.0000e-03, 2.0000e-03],
        [3.0000e+00, 3.0000e+00, 3.0000e-01, 3.0000e-01, 3.0000e-02, 3.0000e-02,
         3.0000e-03, 3.0000e-03]])


<a name='1-2'></a>
### 1.2 - 正弦与余弦位置编码 (Sine and Cosine Positional Encodings)

现在，你可以使用之前计算的角度来计算正弦和余弦位置编码。

$$
PE_{(pos, 2i)}= \sin\left(\frac{pos}{{10000}^{\frac{2i}{d}}}\right)
$$
<br>
$$
PE_{(pos, 2i+1)}= \cos\left(\frac{pos}{{10000}^{\frac{2i}{d}}}\right)
$$

<a name='ex-2'></a>
### 练习 2 - positional_encoding

实现函数 `positional_encoding()` 来计算正弦和余弦位置编码。

**提醒：** 当 $i$ 为偶数时使用正弦公式，当 $i$ 为奇数时使用余弦公式。

#### 额外提示
* 你可能会根据实现方式使用 [np.newaxis](https://numpy.org/doc/stable/reference/arrays.indexing.html)。


In [6]:
def positional_encoding(positions, d_model):
    angle_rads = get_angles(
        torch.arange(positions).unsqueeze(1).float(),
        torch.arange(d_model).unsqueeze(0).float(),
        d_model
    )
    # sin on even indices; cos on odd indices
    angle_rads[:, 0::2] = torch.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = torch.cos(angle_rads[:, 1::2])
    return angle_rads.unsqueeze(0)  # (1, positions, d_model)


很好，你已经成功计算出位置编码！现在，你可以将它们可视化。


In [8]:
# 测试位置编码
pos_encoding = positional_encoding(50, 512)
assert pos_encoding.shape == (1, 50, 512)
print (pos_encoding.shape)
print("get_angles & positional_encoding passed!")

torch.Size([1, 50, 512])
get_angles & positional_encoding passed!


每一行代表一个位置编码——注意每一行都不相同！你为每个单词创建了独特的位置编码。


<a name='2'></a>
## 2 - 掩码 (Masking)

在构建 Transformer 网络时，有两种类型的掩码非常有用：*padding mask*（填充掩码）和 *look-ahead mask*（前瞻掩码）。它们都可以帮助 softmax 计算为输入句子中的词分配合适的权重。

<a name='2-1'></a>
### 2.1 - 填充掩码 (Padding Mask)

通常情况下，你的输入序列可能超过网络能够处理的最大长度。假设模型的最大长度为 5，它输入如下序列：



    [["Do", "you", "know", "when", "Jane", "is", "going", "to", "visit", "Africa"], 
     ["Jane", "visits", "Africa", "in", "September" ],
     ["Exciting", "!"]
    ]


向量化后可能变成：



    [[ 71, 121, 4, 56, 99, 2344, 345, 1284, 15],
     [ 56, 1285, 15, 181, 545],
     [ 87, 600]
    ]
    

当将序列传入 Transformer 模型时，序列长度必须统一。可以通过在序列末尾填充零或截断超过最大长度的句子来实现：



    [[ 71, 121, 4, 56, 99],
     [ 2344, 345, 1284, 15, 0],
     [ 56, 1285, 15, 181, 545],
     [ 87, 600, 0, 0, 0],
    ]
    

长度超过最大值的序列会被截断，而截断后的序列末尾会加上零以保持统一长度。对于长度不足的序列，也会在末尾填充零。然而，这些零会影响 softmax 的计算——这时填充掩码就派上用场了！你需要定义一个布尔掩码，指定哪些元素需要关注(1)，哪些元素需要忽略(0)。之后，你可以使用该掩码将序列中的零设置为接近负无穷大（-1e9）的值。我们会为你实现这部分逻辑，让你可以尽快进入构建 Transformer 网络的乐趣 😇。只需确保理解代码，这样在构建模型时才能正确实现填充。

掩码处理后，输入 `[87, 600, 0, 0, 0]` 会变成 `[87, 600, -1e9, -1e9, -1e9]`，这样在计算 softmax 时，零就不会影响得分。

Keras 中实现的 [MultiheadAttention](https://keras.io/api/layers/attention_layers/multi_head_attention/) 层使用了这种掩码逻辑。


In [9]:
# ----------------------------
# Mask
# ----------------------------
def create_padding_mask(seq):
    mask = (seq == 0).float()
    return mask.unsqueeze(1)  # (batch_size, 1, seq_len)


In [11]:
x = torch.tensor([[7, 6, 0, 0, 1],[1, 2, 3, 0, 0],[0, 0, 0, 4, 5]])
pad_mask = create_padding_mask(x)
assert pad_mask.shape == (3,1,5)
print(create_padding_mask(x))
print("Padding_Mask functions passed!")


tensor([[[0., 0., 1., 1., 0.]],

        [[0., 0., 0., 1., 1.]],

        [[1., 1., 1., 0., 0.]]])
Padding_Mask functions passed!


如果我们将 `(1 - mask)` 乘以 `-1e9` 并加到示例输入序列中，序列中的零就会被设置为接近负无穷大。注意原始序列与掩码序列在计算 softmax 时的区别：


In [13]:
# ----------------------------
# Softmax 测试
# ----------------------------
x_float = x.float()  # 先转换为浮点数

# 普通 softmax
softmax_x = F.softmax(x_float, dim=-1)
print("Softmax without mask:")
print(softmax_x)

# 带 padding mask 的 softmax
# mask 为 1 的位置是被 mask，需要用大负数惩罚
masked_x = x_float + (pad_mask - 1) * 1e9  # pad_mask==1 -> -1e9， pad_mask==0 -> 0
softmax_masked_x = F.softmax(masked_x, dim=-1)
print("Softmax with padding mask:")
print(softmax_masked_x)

Softmax without mask:
tensor([[7.2877e-01, 2.6810e-01, 6.6455e-04, 6.6455e-04, 1.8064e-03],
        [8.4437e-02, 2.2952e-01, 6.2391e-01, 3.1063e-02, 3.1063e-02],
        [4.8541e-03, 4.8541e-03, 4.8541e-03, 2.6503e-01, 7.2041e-01]])
Softmax with padding mask:
tensor([[[0.0000e+00, 0.0000e+00, 5.0000e-01, 5.0000e-01, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 9.5257e-01, 4.7426e-02, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 1.7986e-02, 9.8201e-01, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00, 2.6894e-01, 7.3106e-01],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 5.0000e-01, 5.0000e-01],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 2.6894e-01, 7.3106e-01]],

        [[7.3057e-01, 2.6876e-01, 6.6620e-04, 0.0000e+00, 0.0000e+00],
         [9.0031e-02, 2.4473e-01, 6.6524e-01, 0.0000e+00, 0.0000e+00],
         [3.3333e-01, 3.3333e-01, 3.3333e-01, 0.0000e+00, 0.0000e+00]]])


<a name='2-2'></a>
### 2.2 - 前瞻掩码 (Look-ahead Mask)

前瞻掩码的原理类似。在训练时，你可以访问训练样本的完整正确输出。前瞻掩码帮助模型模拟“已正确预测部分输出”，然后检查在**不看未来信息**的情况下，是否能正确预测下一个输出。

例如，如果期望的正确输出是 `[1, 2, 3]`，你想要检查在模型已正确预测第一个值的情况下，能否预测第二个值，那么就会将第二个和第三个值屏蔽掉。输入的掩码序列为 `[1, -1e9, -1e9]`，然后观察模型是否能生成 `[1, 2, -1e9]`。

因为你努力了，我们也会为你实现这个掩码 😇😇。再次提醒，请仔细查看代码，这样你以后才能有效地实现它。


In [14]:
def create_look_ahead_mask(size):
    mask = torch.triu(torch.ones(size, size), diagonal=1)
    return mask  # (seq_len, seq_len)

In [16]:
# ----------------------------
# 测试 create_look_ahead_mask
# ----------------------------
seq_len = 5
look_ahead_mask = create_look_ahead_mask(seq_len)
assert look_ahead_mask.shape == (seq_len, seq_len)
print("Look-Ahead Mask:")
print(look_ahead_mask)
print("Look-Ahead Mask function passed!")

Look-Ahead Mask:
tensor([[0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]])
Look-Ahead Mask function passed!


<a name='3'></a>
## 3 - 自注意力 (Self-Attention)

正如 Transformer 论文的作者所说，“Attention is All You Need”。

<img src="self-attention.png" alt="Encoder" width="600"/>
<caption><center><font color='purple'><b>图 1: 自注意力计算可视化</b></font></center></caption>

自注意力与传统卷积网络结合使用可以实现并行处理，从而加快训练速度。你将实现 **缩放点积注意力 (scaled dot-product attention)**，它接受查询 (query)、键 (key)、值 (value) 以及掩码 (mask) 作为输入，并返回序列中每个单词的丰富的基于注意力的向量表示。其数学表达式为：
$$
\text{Attention}(Q, K, V) = \operatorname{softmax}\left(\frac{Q K^{T}}{\sqrt{d_k}} + M \right) V \tag{4}
$$

* $Q$ 是查询矩阵 (queries matrix)
* $K$ 是键矩阵 (keys matrix)
* $V$ 是值矩阵 (values matrix)
* $M$ 是可选的掩码
* $d_k$ 是键的维度，用于缩放，以防 softmax 值过大

<a name='ex-3'></a>
### 练习 3 - scaled_dot_product_attention 

实现函数 `scaled_dot_product_attention()` 来创建基于注意力的表示。

**提示**：布尔掩码参数可以传入 `None`，或者是 padding 掩码或前瞻掩码。  

在应用 softmax 之前，将 `(1. - mask)` 乘以 `-1e9`。

#### 额外提示
* 你可能会发现 [tf.matmul](https://www.tensorflow.org/api_docs/python/tf/linalg/matmul) 对矩阵乘法非常有用。


In [17]:

# ----------------------------
# Scaled Dot-Product Attention
# ----------------------------
def scaled_dot_product_attention(q, k, v, mask=None):
    dk = q.size(-1)
    scores = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(dk)
    if mask is not None:
        scores = scores.masked_fill(mask==1, float('-inf'))
    attn = F.softmax(scores, dim=-1)
    output = torch.matmul(attn, v)
    return output, attn



In [20]:
# ----------------------------
# 测试 Scaled Dot-Product Attention
# ----------------------------
q = k = v = torch.rand(2, 3, 4)  # batch_size=2, seq_len=3, depth=4
out, attn = scaled_dot_product_attention(q, k, v)

print("q:\n", q)
print("k:\n", k)
print("v:\n", v)
print("Attention Scores (softmax):\n", attn)
print("Output:\n", out)

assert out.shape == (2, 3, 4) and attn.shape == (2, 3, 3)
print("Scaled Dot-Product Attention passed!")

# 测试加 mask
mask = torch.tensor([[[0,1,0],[0,0,1],[1,0,0]], [[0,0,1],[1,0,0],[0,1,0]]], dtype=torch.uint8)
out_masked, attn_masked = scaled_dot_product_attention(q, k, v, mask=mask)

print("\nMask:\n", mask)
print("Attention Scores with mask (softmax):\n", attn_masked)
print("Output with mask:\n", out_masked)

assert out_masked.shape == (2, 3, 4) and attn_masked.shape == (2, 3, 3)
print("Scaled Dot-Product Attention with mask passed!")

q:
 tensor([[[0.8200, 0.5481, 0.7669, 0.6045],
         [0.2528, 0.8482, 0.1522, 0.5363],
         [0.2815, 0.8123, 0.2681, 0.8129]],

        [[0.4784, 0.7652, 0.7037, 0.8443],
         [0.4885, 0.1822, 0.9119, 0.6112],
         [0.5726, 0.5345, 0.4922, 0.1238]]])
k:
 tensor([[[0.8200, 0.5481, 0.7669, 0.6045],
         [0.2528, 0.8482, 0.1522, 0.5363],
         [0.2815, 0.8123, 0.2681, 0.8129]],

        [[0.4784, 0.7652, 0.7037, 0.8443],
         [0.4885, 0.1822, 0.9119, 0.6112],
         [0.5726, 0.5345, 0.4922, 0.1238]]])
v:
 tensor([[[0.8200, 0.5481, 0.7669, 0.6045],
         [0.2528, 0.8482, 0.1522, 0.5363],
         [0.2815, 0.8123, 0.2681, 0.8129]],

        [[0.4784, 0.7652, 0.7037, 0.8443],
         [0.4885, 0.1822, 0.9119, 0.6112],
         [0.5726, 0.5345, 0.4922, 0.1238]]])
Attention Scores (softmax):
 tensor([[[0.4125, 0.2747, 0.3128],
         [0.3274, 0.3243, 0.3483],
         [0.3350, 0.3130, 0.3520]],

        [[0.4126, 0.3227, 0.2646],
         [0.3699, 0.3601, 0.270

干得好！你现在已经可以实现自注意力了。掌握自注意力后，你就可以开始构建编码器 (encoder) 模块了！


<a name='4'></a>
## 4 - 编码器 (Encoder)

Transformer 编码器层将自注意力 (self-attention) 与卷积神经网络风格的处理结合，以提升训练速度，并将 K 和 V 矩阵传递给解码器 (Decoder)，该解码器将在作业后面构建。在本节作业中，你将通过结合多头注意力和前馈神经网络来实现编码器 (如图 2a 所示)。

<img src="encoder_layer.png" alt="Encoder" width="250"/>
<caption><center><font color='purple'><b>图 2a：Transformer 编码器层</b></font></center></caption>

* `MultiHeadAttention` 可以理解为多次计算自注意力，用于检测不同特征。
* 前馈神经网络 (Feed Forward Neural Network) 包含两个 Dense 层，我们将其实现为 `FullyConnected` 函数。

输入句子首先通过*多头注意力层*，编码器在编码某个单词时会查看输入句子中的其他单词。多头注意力层的输出随后传入*前馈神经网络*。完全相同的前馈网络会独立应用于每个位置。

* 对于 `MultiHeadAttention` 层，你将使用 [Keras 实现](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MultiHeadAttention)。如果你想了解如何将查询矩阵 Q、键矩阵 K 和值矩阵 V 拆分成不同的 head，可以查看实现细节。
* 你还将使用 [Sequential API](https://keras.io/api/models/sequential/) 搭建两个 Dense 层构成的前馈神经网络。


In [21]:
# ----------------------------
# 前馈网络
# ----------------------------
class FullyConnected(nn.Module):
    def __init__(self, d_model, dff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, dff)
        self.linear2 = nn.Linear(dff, d_model)
    def forward(self, x):
        return self.linear2(F.relu(self.linear1(x)))


In [23]:
# ----------------------------
# 测试前馈网络
# ----------------------------
fc = FullyConnected(8, 16)
x = torch.rand(2, 5, 8)  # batch_size=2, seq_len=5, d_model=8

# 前向传播
y = fc(x)

# 打印数字输出
print("Input x:\n", x)
x1 = F.relu(fc.linear1(x))
print("After linear1 + ReLU:\n", x1)
print("Output y:\n", y)

# 验证输出形状
assert y.shape == x.shape
print("FullyConnected passed!")

Input x:
 tensor([[[0.3513, 0.2513, 0.2308, 0.1186, 0.6873, 0.3637, 0.5210, 0.4875],
         [0.0917, 0.8506, 0.3968, 0.9119, 0.2783, 0.7048, 0.6517, 0.4378],
         [0.8261, 0.2454, 0.1647, 0.0172, 0.2447, 0.1091, 0.6365, 0.1570],
         [0.2795, 0.8758, 0.7508, 0.1421, 0.8066, 0.9451, 0.3369, 0.8978],
         [0.1600, 0.8795, 0.0888, 0.0454, 0.8192, 0.8937, 0.1246, 0.6692]],

        [[0.6951, 0.4703, 0.2725, 0.7294, 0.4237, 0.1618, 0.6358, 0.5917],
         [0.9310, 0.5374, 0.6483, 0.4942, 0.1652, 0.0818, 0.0631, 0.5288],
         [0.3806, 0.7728, 0.7665, 0.2217, 0.5649, 0.3021, 0.7523, 0.3003],
         [0.5093, 0.6719, 0.2087, 0.4812, 0.4554, 0.5704, 0.2378, 0.1722],
         [0.1584, 0.4287, 0.4379, 0.4206, 0.9504, 0.8533, 0.6630, 0.7453]]])
After linear1 + ReLU:
 tensor([[[0.2839, 0.0000, 0.0560, 0.0479, 0.0000, 0.0000, 0.0244, 0.3960,
          0.0000, 0.0000, 0.1330, 0.0000, 0.0000, 0.1323, 0.0000, 0.4261],
         [0.3787, 0.1663, 0.4007, 0.6023, 0.0000, 0.0000, 0.0000

<a name='4-1'></a>
### 4.1 编码器层 (Encoder Layer)

现在你可以在一个编码器层中将多头注意力和前馈神经网络结合起来！你还将使用残差连接 (residual connections) 和层归一化 (layer normalization) 来加速训练 (图 2a)。

<a name='ex-4'></a>
### 练习 4 - EncoderLayer

使用 `call()` 方法实现 `EncoderLayer()`。

在本练习中，你将实现一个编码器块（图 2），使用 `call()` 方法。该函数应执行以下步骤：

1. 将 Q、V、K 矩阵以及布尔掩码 (boolean mask) 输入多头注意力层。请记住，为了计算*自注意力 (self-attention)*，Q、V 和 K 应相同。让 `return_attention_scores` 和 `training` 使用默认值。在训练过程中，还需在多头注意力层中执行 Dropout。
2. 添加跳跃连接 (skip connection)，将原始输入 `x` 与多头注意力层输出相加。
3. 添加跳跃连接后，将输出传入第一个归一化层 (Layer Normalization)。
4. 最后重复步骤 1-3，但使用前馈神经网络和 Dropout 层代替多头注意力层。

<details>
  <summary><font size="2" color="darkgreen"><b>额外提示 (点击展开)</b></font></summary>
    
* `__init__` 方法创建所有将在 `call` 方法中访问的层。无论何时在 `call` 方法中使用 `__init__` 中定义的层，都必须使用语法 `self.[层名]`。
* 可以查阅 [MultiHeadAttention 文档](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MultiHeadAttention) 获取帮助。*注意，如果 query、key 和 value 相同，则该函数执行自注意力 (self-attention)。*
* `self.mha` 的 call 参数如下（B 为 batch_size，T 为目标序列长度，S 为输出长度）：
  - `query`：形状为 (B, T, dim) 的查询张量
  - `value`：形状为 (B, S, dim) 的值张量
  - `key`：可选，形状为 (B, S, dim) 的键张量。如果未提供，则使用 value 作为 key 和 value（最常用情况）。
  - `attention_mask`：形状为 (B, T, S) 的布尔掩码，防止注意力关注某些位置。布尔掩码指定查询元素可以关注哪些键元素，1 表示关注，0 表示不关注。缺失的 batch 和 head 维度可自动广播。
  - `return_attention_scores`：布尔值，指示是否返回注意力输出。如果为 True，则只返回注意力输出；否则返回 (attention_output, attention_scores)。默认值为 False。
  - `training`：布尔值，指示层是否处于训练模式（添加 dropout）或推理模式（不添加 dropout）。默认值为父模型/层的训练模式，如果无父层，则默认为 False（推理模式）。
</details>


In [24]:
# ----------------------------
# Encoder Layer
# ----------------------------
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, dropout=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        self.ffn = FullyConnected(d_model, dff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, mask=None):
        attn_out,_ = self.mha(x,x,x, key_padding_mask=mask)
        x = self.norm1(x + attn_out)
        ffn_out = self.dropout(self.ffn(x))
        x = self.norm2(x + ffn_out)
        return x


In [26]:
# ----------------------------
# 测试 EncoderLayer
# ----------------------------
enc_layer = EncoderLayer(8, 2, 16)
x = torch.rand(2,5,8)  # batch_size=2, seq_len=5, d_model=8
mask = torch.zeros(2,5)  # 没有padding
out = enc_layer(x, mask)

print("Output of EncoderLayer:\n", out)
assert out.shape == x.shape
print("EncoderLayer passed!")

Output of EncoderLayer:
 tensor([[[ 0.0314,  0.5726,  0.5395,  0.2765, -0.3497, -1.3844, -1.4577,
           1.7719],
         [-0.2994,  0.6061,  1.0795,  1.2569, -0.8556, -1.8855, -0.4538,
           0.5518],
         [ 0.2667, -1.8838, -0.0271,  1.4896,  0.2043, -0.0229, -1.0424,
           1.0156],
         [ 1.3583, -1.1074, -0.4414,  1.0912, -1.4507, -0.6820,  0.2888,
           0.9433],
         [-0.7208,  0.4611,  0.8476, -0.9587,  0.0526,  1.6104, -1.6948,
           0.4026]],

        [[ 1.4858,  1.4418, -0.4724, -0.4241, -1.5317,  0.5932, -0.4577,
          -0.6348],
         [ 1.7666, -1.2154, -1.0147, -0.0202, -1.1960,  0.5424,  0.5811,
           0.5563],
         [ 1.1912, -0.2158,  0.6246,  0.7333, -2.0517,  0.0110, -0.9691,
           0.6766],
         [-0.5451, -0.1484,  0.1374,  0.4691, -1.2809, -0.7139, -0.2088,
           2.2908],
         [ 0.6832, -1.6518, -0.3179,  1.1011, -1.2169,  1.1806, -0.4332,
           0.6549]]], grad_fn=<NativeLayerNormBackward0>)
Encod

<a name='4-2'></a>
### 4.2 - 完整编码器 (Full Encoder)

太棒了！你现在已经成功实现了位置编码 (positional encoding)、自注意力 (self-attention) 和编码器层 (encoder layer) —— 给自己一个鼓励的掌声👏。现在你准备好构建完整的 Transformer 编码器（图 2b）了，你将对输入进行嵌入，并添加你计算好的位置编码。然后将编码后的嵌入送入多层编码器堆栈中。

<img src="encoder.png" alt="Encoder" width="330"/>
<caption><center><font color='purple'><b>图 2b: Transformer 编码器</b></font></center></caption>

<a name='ex-5'></a>
### 练习 5 - Encoder

使用 `call()` 方法完成 `Encoder()` 函数，以对输入进行嵌入、添加位置编码，并实现多个编码器层。

在本练习中，你将用 Embedding 层、位置编码以及多个 EncoderLayer 初始化编码器。你的 `call()` 方法应执行以下步骤：

1. 将输入传入 Embedding 层。
2. 将嵌入进行缩放 (scale)，通过乘以嵌入维度的平方根来实现。记得在计算平方根前将嵌入维度转换为 `tf.float32` 数据类型。
3. 添加位置编码：`self.pos_encoding[:, :seq_len, :]` 加到嵌入上。
4. 将编码后的嵌入传入 dropout 层，记得使用 `training` 参数设置模型的训练模式。
5. 使用 for 循环将 dropout 层的输出传入编码器堆栈中的多层编码器。


In [28]:
# ----------------------------
# Encoder
# ----------------------------
class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, max_pos_enc):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(max_pos_enc, d_model)
        self.layers = nn.ModuleList([EncoderLayer(d_model,num_heads,dff) for _ in range(num_layers)])
        self.dropout = nn.Dropout(0.1)
    def forward(self, x, mask=None):
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = x + self.pos_encoding[:,:x.size(1), :]
        x = self.dropout(x)
        for layer in self.layers:
            x = layer(x, mask)
        return x



In [29]:
# ----------------------------
# 测试 Encoder
# ----------------------------
encoder = Encoder(2,8,2,16,50,100)
x_inp = torch.randint(0,50,(2,10))  # batch_size=2, seq_len=10
out = encoder(x_inp)
print("Final Encoder output:\n", out)
assert out.shape == (2,10,8)
print("Encoder passed!")

Final Encoder output:
 tensor([[[-1.2887,  1.1822,  0.1434,  1.1508, -1.2586,  1.1050, -0.1589,
          -0.8753],
         [-1.6305, -0.4302,  0.3122,  0.4399, -1.4670,  1.0302,  0.6190,
           1.1265],
         [ 1.5993, -0.0456,  0.2878, -1.2994, -1.0969,  1.4314, -0.5660,
          -0.3106],
         [-0.5984,  0.7124, -1.9254, -0.3253,  1.0719,  1.4347, -0.3355,
          -0.0342],
         [-0.9853,  0.8708,  1.1447,  1.3021, -1.7555, -0.3262, -0.2760,
           0.0254],
         [-1.1708, -0.4520,  1.6188,  1.5009, -0.7907,  0.1383,  0.1024,
          -0.9470],
         [-1.2659,  2.1031, -0.0899,  0.3234, -0.0807, -0.3000, -1.2189,
           0.5288],
         [-1.0928,  1.4618, -1.8016,  0.0960,  0.8019,  0.7824, -0.3770,
           0.1292],
         [-0.8542,  0.4746,  0.0064,  1.5150, -0.9984,  1.2456,  0.0918,
          -1.4809],
         [-0.2980,  1.2445, -0.5708,  1.5310, -0.2904,  0.6579, -0.5943,
          -1.6799]],

        [[-0.4503, -0.2007,  0.7637,  0.7922,

<a name='5'></a>
## 5 - 解码器 (Decoder)

解码器层接收编码器生成的 K 和 V 矩阵，并使用输出的 Q 矩阵计算第二个多头注意力层 (multi-head attention)（图 3a）。

<img src="decoder_layer.png" alt="Encoder" width="250"/>
<caption><center><font color='purple'><b>图 3a: Transformer 解码器层</b></font></center></caption>

<a name='5-1'></a>    
### 5.1 - 解码器层 (Decoder Layer)

同样，你将多头注意力与前馈神经网络结合，但这次需要实现两层多头注意力层。你还会使用残差连接 (residual connections) 和层归一化 (layer normalization) 来加快训练速度（图 3a）。

<a name='ex-6'></a>    
### 练习 6 - DecoderLayer

使用 `call()` 方法实现 `DecoderLayer()`：

1. **Block 1** 是带有残差连接和前瞻掩码 (look-ahead mask) 的多头注意力层。与 `EncoderLayer` 类似，Dropout 在多头注意力层中定义。
2. **Block 2** 会考虑编码器的输出，因此多头注意力层将接收来自编码器的 K 和 V，以及来自 Block 1 的 Q。然后像在 `EncoderLayer` 中一样，应用归一化层和残差连接。
3. **Block 3** 是一个前馈神经网络，带有 Dropout、归一化层和残差连接。
    
**额外提示：**
* 前两个块与 `EncoderLayer` 类似，不过在计算自注意力时，需要返回 `attention_scores`。


In [30]:
# ----------------------------
# Decoder Layer
# ----------------------------
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff):
        super().__init__()
        self.mha1 = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        self.mha2 = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        self.ffn = FullyConnected(d_model,dff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)
    def forward(self, x, enc_output, look_ahead_mask=None, padding_mask=None):
        attn1,_ = self.mha1(x,x,x,key_padding_mask=look_ahead_mask)
        x = self.norm1(x + attn1)
        attn2,_ = self.mha2(x, enc_output, enc_output, key_padding_mask=padding_mask)
        x = self.norm2(x + attn2)
        ffn_out = self.dropout(self.ffn(x))
        x = self.norm3(x + ffn_out)
        return x


In [31]:
# ----------------------------
# 测试 DecoderLayer
# ----------------------------
dec_layer = DecoderLayer(8,2,16)
enc_out = torch.rand(2,10,8)   # Encoder输出(batch_size=2, seq_len=10, d_model=8)
x_tar = torch.rand(2,5,8)      # Decoder输入(batch_size=2, target_seq_len=5, d_model=8)
out = dec_layer(x_tar, enc_out)
print("Final DecoderLayer output:\n", out)
assert out.shape == x_tar.shape
print("DecoderLayer passed!")

Final DecoderLayer output:
 tensor([[[ 0.2048, -1.4163,  1.7223,  1.0603, -0.3060,  0.1882, -0.1452,
          -1.3082],
         [-0.7742, -1.2328,  1.2440,  0.9438,  0.0511, -0.8935,  1.4314,
          -0.7698],
         [ 0.0073, -0.2614,  1.8183,  1.2853, -0.6405, -0.3329, -0.3497,
          -1.5264],
         [ 0.8275,  0.4722,  2.0929, -0.4023, -1.1409, -0.4757, -0.4880,
          -0.8855],
         [-1.4754, -0.2938,  1.5280,  1.0081, -1.2989,  0.3728,  0.6024,
          -0.4432]],

        [[-1.2651,  0.4915,  1.2030,  1.3381,  0.0392,  0.2664, -0.4455,
          -1.6277],
         [-0.9654, -1.8488,  1.3095,  0.7773, -0.1100,  0.8894,  0.4869,
          -0.5388],
         [ 0.6265, -0.8522,  0.8081, -0.0264, -1.2837,  1.9259, -0.8751,
          -0.3232],
         [ 0.0217,  0.3204,  1.1604,  0.8521, -1.2287,  0.2628,  0.5864,
          -1.9753],
         [-0.6750,  0.5070,  2.2912,  0.2357, -1.1515, -0.1208, -0.7032,
          -0.3834]]], grad_fn=<NativeLayerNormBackward0>)
De

<a name='5-2'></a> 
### 5.2 - 完整解码器 (Full Decoder)

你已经接近完成了！现在使用解码器层 (DecoderLayer) 来构建完整的 Transformer 解码器（图 3b）。你需要对输出进行嵌入 (embedding) 并添加位置编码，然后将编码后的嵌入输入到多层解码器堆栈中。

<img src="decoder.png" alt="Encoder" width="300"/>
<caption><center><font color='purple'><b>图 3b: Transformer 解码器</b></font></center></caption>

<a name='ex-7'></a>     
### 练习 7 - Decoder

使用 `call()` 方法实现 `Decoder()`，对输出进行嵌入、添加位置编码，并实现多层解码器。

在本练习中，你将初始化解码器，包含嵌入层、位置编码和多个 DecoderLayers。你的 `call()` 方法将执行以下步骤：
1. 将生成的输出通过嵌入层 (Embedding layer)。
2. 将嵌入向量按嵌入维度的平方根进行缩放。记得在计算平方根前，将嵌入维度转换为 `tf.float32` 类型。
3. 添加位置编码：`self.pos_encoding[:, :seq_len, :]` 到嵌入向量中。
4. 将编码后的嵌入通过 Dropout 层，使用 `training` 参数设置模型的训练模式。
5. 使用 for 循环将 Dropout 层输出传入解码器层堆栈 (DecoderLayers)。


In [32]:
# ----------------------------
# Decoder
# ----------------------------
class Decoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, max_pos_enc):
        super().__init__()
        self.embedding = nn.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(max_pos_enc, d_model)
        self.layers = nn.ModuleList([DecoderLayer(d_model,num_heads,dff) for _ in range(num_layers)])
        self.dropout = nn.Dropout(0.1)
    def forward(self, x, enc_output):
        x = self.embedding(x) * math.sqrt(self.embedding.embedding_dim)
        x = x + self.pos_encoding[:,:x.size(1), :]
        x = self.dropout(x)
        for layer in self.layers:
            x = layer(x, enc_output)
        return x


In [33]:
# ----------------------------
# 测试 Decoder (数字输出)
# ----------------------------
decoder = Decoder(2, 8, 2, 16, 50, 100)
enc_out = torch.rand(2,10,8)  # 假设 Encoder 输出
x_tar = torch.randint(0,50,(2,5))
out = decoder(x_tar, enc_out)
print("\nFinal Decoder output:\n", out)
assert out.shape == (2,5,8)
print("Decoder passed!")


Final Decoder output:
 tensor([[[ 0.1341,  0.6275,  0.7025,  0.5792, -1.8678, -1.5213,  0.8322,
           0.5136],
         [ 0.7185,  0.0637, -0.2469,  1.0986, -1.2498, -1.2965,  1.5859,
          -0.6736],
         [ 0.3267,  0.2996,  0.8264,  0.6075, -1.9368, -1.4355,  0.8541,
           0.4580],
         [-0.2054, -1.8874, -1.1054, -0.1506,  0.7862,  0.4343,  1.2630,
           0.8653],
         [-0.5874, -2.0008, -0.2940, -0.4756,  0.9730,  0.2215,  1.1216,
           1.0418]],

        [[ 0.1709,  0.6166, -0.1811, -0.3149, -1.4836, -1.3867,  1.2195,
           1.3592],
         [-1.2746, -0.7594, -1.5833,  1.1744,  0.3318,  1.1219,  0.3296,
           0.6597],
         [-0.0075,  0.0063, -1.7500,  0.3403, -0.8690, -0.3657,  0.8574,
           1.7882],
         [-1.4265, -0.6063,  0.9315,  1.2528,  0.5386, -0.1303, -1.4410,
           0.8812],
         [-0.3214,  0.0809, -1.6887,  0.5750, -0.3769, -0.7176,  0.4989,
           1.9499]]], grad_fn=<NativeLayerNormBackward0>)
Decode

<a name='6'></a> 
## 6 - Transformer

呼！这次作业真的很长，现在你已经来到了深度学习专项课程的最后一个练习。恭喜！你已经完成了所有的核心工作，现在是时候把所有模块整合起来了。  

<img src="transformer.png" alt="Transformer" width="550"/>
<caption><center><font color='purple'><b>图 4: Transformer</b></font></center></caption>
    
Transformer 架构的数据流如下：
* 首先，输入通过编码器 (Encoder)，也就是你实现的多个 Encoder 层的堆叠：
    - 输入的嵌入和位置编码
    - 对输入的多头注意力 (Multi-Head Attention)
    - 前馈神经网络 (Feed Forward Neural Network) 用于特征提取
* 然后，预测输出通过解码器 (Decoder)，由你实现的 Decoder 层组成：
    - 输出的嵌入和位置编码
    - 对生成输出的多头注意力
    - 使用来自第一个多头注意力层的 Q，以及编码器输出的 K 和 V 进行多头注意力
    - 前馈神经网络帮助提取特征
* 最后，在第 N 个解码器层之后，应用两个全连接层和 softmax 来生成序列中下一个输出的预测。

<a name='ex-8'></a> 
### 练习 8 - Transformer

使用 `call()` 方法实现 `Transformer()`：
1. 将输入通过编码器，并使用适当的掩码 (mask)。
2. 将编码器输出和目标序列通过解码器，并使用适当的掩码。
3. 使用线性变换 (linear transformation) 和 softmax 生成预测。


In [35]:
# ----------------------------
# Transformer
# ----------------------------
class Transformer(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, max_pos_enc_input, max_pos_enc_target):
        super().__init__()
        self.encoder = Encoder(num_layers,d_model,num_heads,dff,input_vocab_size,max_pos_enc_input)
        self.decoder = Decoder(num_layers,d_model,num_heads,dff,target_vocab_size,max_pos_enc_target)
        self.final_layer = nn.Linear(d_model,target_vocab_size)
    def forward(self, inp, tar):
        enc_out = self.encoder(inp)
        dec_out = self.decoder(tar, enc_out)
        return F.softmax(self.final_layer(dec_out), dim=-1)



In [37]:
# ----------------------------
# 测试 Transformer（增加数字输出）
# ----------------------------
model = Transformer(2,8,2,16,50,50,100,100)
x_inp = torch.randint(0,50,(2,10))
x_tar = torch.randint(0,50,(2,5))
print("Input sequence indices (x_inp):", x_inp)
print("Target sequence indices (x_tar):", x_tar)

out = model(x_inp,x_tar)
print("\nTransformer output shape:", out.shape)
print("Transformer output sample values (first batch, first 3 tokens):")
print(out[0,:3,:5])  # 显示第一条序列前三个位置，前五个词的概率

# 打印数值统计信息
print("\nOutput statistics:")
print("Max value:", out.max().item())
print("Min value:", out.min().item())
print("Mean value:", out.mean().item())
print("Sum along vocab dimension (should be 1 for softmax):", out[0,0,:].sum().item())

print("\nTransformer passed!")


Input sequence indices (x_inp): tensor([[37, 38, 30, 31,  0, 49, 49, 48, 40, 35],
        [ 7, 18, 12, 31, 21, 42, 33, 27, 36, 33]])
Target sequence indices (x_tar): tensor([[ 9, 17, 19, 21, 24],
        [33, 47, 42, 21, 19]])

Transformer output shape: torch.Size([2, 5, 50])
Transformer output sample values (first batch, first 3 tokens):
tensor([[0.0089, 0.0054, 0.0122, 0.0095, 0.0430],
        [0.0427, 0.0088, 0.0198, 0.0269, 0.0178],
        [0.0453, 0.0128, 0.0333, 0.0264, 0.0131]], grad_fn=<SliceBackward0>)

Output statistics:
Max value: 0.07378123700618744
Min value: 0.00341250142082572
Mean value: 0.019999999552965164
Sum along vocab dimension (should be 1 for softmax): 1.0

Transformer passed!


## 结论

你已经完成了本次作业的评分部分。到目前为止，你已经掌握了以下内容：

* 创建位置编码 (Positional Encodings) 来捕捉数据中的序列关系
* 使用词嵌入 (Word Embeddings) 计算缩放点积自注意力 (Scaled Dot-Product Self-Attention)
* 实现带掩码的多头注意力 (Masked Multi-Head Attention)
* 构建并训练一个 Transformer 模型


<font color='blue'>
    <b>你需要记住的内容</b>：

- 自注意力 (Self-Attention) 与卷积网络层 (Convolutional Network Layers) 的结合能够实现训练的并行化，从而加快训练速度。
- 自注意力是通过生成的查询矩阵 Q、键矩阵 K 和值矩阵 V 来计算的。
- 将位置编码 (Positional Encoding) 添加到词嵌入 (Word Embeddings) 中，是在自注意力计算中引入序列信息的有效方法。
- 多头注意力 (Multi-Head Attention) 有助于检测句子中的多个特征。
- 掩码 (Masking) 防止模型在训练过程中“偷看”未来信息，或者在处理截断句子时过度加权零值。


现在你已经完成了 Transformer 作业，别忘了查看非评分实验 (ungraded labs)，将 Transformer 模型应用到实际案例中，比如命名实体识别 (NER) 和问答 (QA) 。

# 恭喜你完成深度学习专项课程！！！！ 🎉🎉🎉🎉🎉

这是该专项课程的最后一个评分作业。现在是时候庆祝你所有的努力和付出了！

<a name='7'></a> 
## 7 - 参考文献

Transformer 算法由 Vaswani 等人提出 (2017)。

- Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin (2017). [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
